# k-means
#### Construstor
- Read data from file.
- Initialize centroids.  

> 
```
def __init__(self, data, ground, cluster_num):
    # read files
    with open(data) as f:
        lines = f.readlines()
    self.points = [[float(x) for x in line.strip().split(" ")] for line in lines]
    with open(ground) as f:
        lines = f.readlines()
    self.labels = [float(line.strip()) for line in lines]
    # init variables
    self.cluster_num = cluster_num
    self.predicts = [0 for i in range(len(self.points))]
    self.centroids = [self.points[idx] for idx in random.sample(range(len(self.points)), cluster_num)]
```

#### Expectation (E) step
- Estimate euclidien distance to every centroid.
- Assign cluster label by minimum distance to centroids.

> 
```
def E_step(self):
    for i in range(len(self.points)):
        point = self.points[i]
        min_distance = sys.maxint
        for j in range(len(self.centroids)):
            centroid = self.centroids[j]
            # Euclidien distance
            distance = math.pow(point[0] - centroid[0], 2) + math.pow(point[1] - centroid[1], 2)
            if distance < min_distance:
                self.predicts[i] = j
                min_distance = distance
```

#### Maximization (M) step
- Calculate new centroids by new cluster labels.

> 
```
def M_step(self):
    # save previous centroids and it will be used to see if centroids moved or not
    prev_centroids = self.centroids
    # sum up all points in the cluster
    self.centroids = [[0.0, 0.0] for i in range(self.cluster_num)]
    count = [0] * self.cluster_num
    for i in range(len(self.points)):
        point = self.points[i]
        self.centroids[self.predicts[i]][0] += point[0]
        self.centroids[self.predicts[i]][1] += point[1]
        count[self.predicts[i]] += 1
    # calculate means by dividing it by number of points in cluster
    for i in range(self.cluster_num):
        if count[i] != 0:
            self.centroids[i][0] /= count[i]
            self.centroids[i][1] /= count[i]
```

#### Initialize method
👉 Randomly select data points to be the centroids.

> 
```
self.centroids = [self.points[idx] for idx in random.sample(range(len(self.points)), cluster_num)]
```

#### Converge
👉 Observe that centroids moved of not.

> 
```
# calculate euclidien distance between last centroids and current centroids
error = 0.0
for i in range(self.cluster_num):
    error += self.centroids[i][0] * prev_centroids[i][0] + self.centroids[i][1] * prev_centroids[i][1]
# if distance is zero, it means centroid didn't move
if error - self.error == 0:
    self.converged = True
self.error = error
```

#### Run and make video
- Use matplotlib.animation.FuncAnimation to generete video

> 
```
ani = animation.FuncAnimation(fig=fig, func=update, frames=generator, interval=300, blit=False)
```

- Function Update wil be called every frames.  
  In update function, it clear whole figure and re-scatter points.  
  In update function, it call E step when it's odd frames, call M step otherwise.
  


- Notice that I use generator to determine how many frames it should be draw

> 
```
def generator():
    n = 0
    # stop generate when model is converged
    while not model.converged:
        yield n
        n += 1
```

# kernel k-means
- RBF kernel $$ exp( - \frac{ \lVert x - x' \rVert^2 }{ 2 \sigma^2 } ) $$

> 
```
def RBF_kernel(self, x1, x2, gamma):
      return math.exp(gamma * (math.pow(x1[0]-x2[0], 2) + math.pow(x1[1]-x2[1], 2)))
```

- Initialize centroids by randomly assign predicts to points, and get A matrix

> 
```
self.predicts = np.zeros((len(self.points), self.cluster_num))
for i in range(len(self.points)):
      self.predicts[i][random.randint(0, self.cluster_num - 1)] = 1
self.cluster_count = np.zeros((self.cluster_num, self.cluster_num))
for predict in self.predicts:
      idx = np.argmax(predict)
      self.cluster_count[idx][idx] += 1.0
for i in range(self.cluster_num):
      self.cluster_count[i][i] = 1.0 / float(self.cluster_count[i][i])
```

- E step: calculate distance on phi(x)
$$ \arg\min_{k} \  \lVert \phi(x) - C_{k} \rVert ^2 = \arg\min_{k} (DA'KAD)_{kk} - 2 (k'AD)_{k} $$
    - D is NxN diagonal matrix with recipocal of number of points in the cluster.
    - A is weight matrix, which ith row is hot-one representation of label respect to ith data point.  
    In my code, self.cluster_count is this A matrix
    - K is gran matrix and be composed of distance calculated by RBF kernel.  
    In my code, self.gran_mat is this gran matrix
    - k is the kth row in gran matrix
    
> 
```
def E_step(self):
      kk = np.dot(self.cluster_count, self.predicts.T)
      kk = np.dot(kk, self.gran_mat)
      kk = np.dot(kk, self.predicts)
      kk = np.dot(kk, self.cluster_count)
      kk = kk.diagonal()
      for point_idx in range(len(self.points)):
          k = np.dot(self.gran_mat[point_idx,:], self.predicts)
          k = np.dot(k, self.cluster_count)
          distance = kk - 2* k
          self.predicts[point_idx, :] = np.zeros(self.cluster_num)
          self.predicts[point_idx, np.argmin(distance)] = 1
```

-  step: update A matrix by label prediction

> 
```
def M_step(self):
    self.cluster_count = np.zeros((self.cluster_num, self.cluster_num))
    for predict in self.predicts:
        idx = np.argmax(predict)
        self.cluster_count[idx][idx] += 1.0         
    for i in range(self.cluster_num):
        self.cluster_count[i][i] = 1.0 / float(self.cluster_count[i][i])
```

# spectral clustering

- Graph Laplacian $$ L = D^{-1/2}AD^{-1/2} $$

> 
```
def calc_laplacian(self, A):
      D = np.diag(np.array([math.pow(np.sum(row), -0.5) for row in A]))
      return D.dot(A).dot(D)
```

- Get eigen vector of Graph Laplacian

> 
```
eig_val, eig_vec = eigs(self.L, k=self.cluster_num)
```

- Re-normalize the eigen vector by rows

> 
```
rows_norm = np.linalg.norm(self.X, axis=1)
self.Y = (self.X.T / rows_norm).T
```

- Run k-means

> 
```
self.predicts = self.k_means(self.Y)
```

# Bonus
### Initialization
- Randomly select data points as centroids
- Randomly assign centroids values
- Randomly assign prediction label to data points, and calculate initial centroids


👉 Top two methods has almost same result.  
👉 The third method spend much time to converge, because the initial centroids are to close to each other. See follow image:
<img src="k_means_test1_random_predict.JPG" width="300">
